In [117]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import logging
import subprocess
import re
import sys
import time
load_dotenv()


True

In [118]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True,  # перезаписывает любые ранние настройки/хендлеры
)

logging.info("Загружены переменные из .env")

logger = logging.getLogger("spotify")

2025-11-12 15:59:01,234 INFO root Загружены переменные из .env


In [119]:
client_id = os.getenv('SPOTIPY_CLIENT_ID') or os.getenv('client_id_sp')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET') or os.getenv('client_secret_sp')

In [120]:
if not client_id or not client_secret:
    logging.error("Отсутствуют client_id или client_secret. Установите SPOTIPY_CLIENT_ID и SPOTIPY_CLIENT_SECRET или client_id_sp/client_secret_sp в .env")
    raise RuntimeError("No client_id or client_secret.")
try:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    logging.info("Аутентификация (Client Credentials) успешно выполнена")
except Exception as e:
    logging.error(f"Ошибка при аутентификации (Client Credentials): {e}")
    raise

2025-11-12 15:59:01,247 INFO root Аутентификация (Client Credentials) успешно выполнена


In [121]:
auth = SpotifyClientCredentials(
    client_id,
    client_secret,
)
sp = spotipy.Spotify(auth_manager=auth)

In [122]:
artists_to_check = pd.read_csv('/Users/anastasiahimic/Desktop/hse_python_project_2/data_scrapping/files/spotify_artists.csv')['Rank']
artists_to_check 

0                  Drake
1           Taylor Swift
2              Bad Bunny
3             The Weeknd
4          Justin Bieber
              ...       
2995        Gilberto Gil
2996                Roar
2997             Redimi2
2998         Jessie Ware
2999    Marco Barrientos
Name: Rank, Length: 3000, dtype: object

In [123]:
def get_artist_labels(sp, artist_names):
    out = {}
    logging.info(f"Старт: {len(artist_names)} артистов в очереди")  
    for idx, name in enumerate(artist_names, 1):
        logging.info(f"[{idx}/{len(artist_names)}] Ищу артиста: {name}")  
        try:
            artist = sp.search(q=f'artist:{name}', type='artist', limit=1)\
                       .get('artists', {}).get('items', [])
            if not artist:
                logging.warning(f"Артист не найден: {name}")  
                out[name] = []
                continue

            artist_id = artist[0]['id']
            logging.info(f"Найден artist_id={artist_id} для {name}")  

            albums = sp.artist_albums(artist_id, album_type='album,single', limit=10)  
            items = albums.get('items', [])
            logging.info(f"У {name} получено {len(items)} альбомов") 

            labels = set()
            for a in items:
                alb = sp.album(a['id'])
                for c in (alb.get('copyrights') or []):
                    txt = c.get('text', '').strip()
                    if txt:
                        labels.add(txt)


            out[name] = sorted(labels)
            logging.info(f"Лейблы для {name}: {out[name]}") 

            time.sleep(0.25)  
        except Exception as e:
            logging.error(f"Ошибка при обработке {name}: {e}", exc_info=True)  
            out[name] = []
    logging.info("Готово: обработка завершена") 
    return out

In [124]:
labels_by_artist = get_artist_labels(sp, artists_to_check.head())  # Тестируем

2025-11-12 15:59:01,285 INFO root Старт: 5 артистов в очереди
2025-11-12 15:59:01,286 INFO root [1/5] Ищу артиста: Drake
2025-11-12 15:59:02,015 INFO root Найден artist_id=3TVXtAsR1Inumwj472S9r4 для Drake
2025-11-12 15:59:02,111 WARNING root Your application has reached a rate/request limit. Retry will occur after: 21171


KeyboardInterrupt: 

In [125]:
data = []
for artist, labels in labels_by_artist.items():
    if labels:
        for label in labels:
            data.append({'Artist': artist, 'Label': label})
labels_df = pd.DataFrame(data)

In [126]:
labels_df

,Artist,Label
0,Drake,Dot Records
1,Drake,"Reverend White, LLC."
2,Drake,River House Artists
3,Drake,"Reverend White, LLC"
4,Taylor Swift,Taylor Swift
5,Bad Bunny,Rimas Entertainment LLC.
6,Bad Bunny,Rimas Entertainment LLC
7,Bad Bunny,Rimas Entertainment
8,Bad Bunny,"UMG Recordings, Inc."
9,The Weeknd,"The Weeknd XO, Inc., manufactured and marketed..."
